In [1]:
from google.colab import drive

drive_path = '/content/drive'
drive.mount(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!7z x /content/drive/MyDrive/2022_security_project1.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 178076816 bytes (170 MiB)

Extracting archive: /content/drive/MyDrive/2022_security_project1.zip
--
Path = /content/drive/MyDrive/2022_security_project1.zip
Type = zip
Physical Size = 178076816

  0%    
Would you like to replace the existing file:
  Path:     ./Security Project1/BCE_model.py
  Size:     2858 bytes (3 KiB)
  Modified: 2022-11-02 01:57:33
with the file from archive:
  Path:     Security Project1/BCE_model.py
  Size:     2858 bytes (3 KiB)
  Modified: 2022-11-02 01:57:33
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? s

  0% 1      Everything is Ok

Folders: 4
Files

In [3]:
%cd "./Security Project1"

/content/Security Project1


In [4]:
from Utils import open_pickle
import pandas as pd
import numpy as np

In [5]:
train_path = r"./train.csv"
valid_path = r"./valid.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [6]:
# 학습, 검증 플로우 데이터 개수 확인
print(len(train_df), len(valid_df))

977270 325560


In [7]:
# 주어진 플로우 컬럼 종류 확인
train_df.keys()

Index(['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Flags',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max'

In [8]:
# 데이터 확인
train_df.head(5)

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,110.15.155.169,13850,97.2.159.1,53,TCP,......S.,03:52:00,3119967,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,110.15.155.169,43842,97.2.159.1,53,TCP,......S.,03:37:20,2991830,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,110.15.155.171,25918,97.2.159.1,53,TCP,......S.,03:10:28,3004658,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,238.61.97.110,4445,97.2.159.1,53,TCP,...A...F,00:42:24,4402645,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,110.15.155.171,28467,97.2.159.1,53,TCP,......S.,02:45:28,3001784,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


In [9]:
#TODO1
train_df["Label"].replace('Benign_IP', 0, inplace=True)
train_df["Label"].replace('Attack_IP', 1, inplace=True)
valid_df["Label"].replace('Benign_IP', 0, inplace=True)
valid_df["Label"].replace('Attack_IP', 1, inplace=True)
train_y = train_df["Label"].values.tolist()
valid_y = valid_df["Label"].values.tolist()

train_list = train_df.drop(columns = ["Src IP", "Dst IP", "Protocol", "Timestamp", "Flags", "Label"]).values.tolist() 
valid_list = valid_df.drop(columns = ["Src IP", "Dst IP", "Protocol", "Timestamp", "Flags", "Label"]).values.tolist()

In [10]:
# Problem 2~를 실행 후에는 Solved가 print되지 않을 수 있음(같은 변수명 공유)
if "answer_train_list" not in locals():
    answer_train_list = open_pickle("./problem1_answer/answer_train_list.pkl")
if "answer_valid_list" not in locals():
    answer_valid_list = open_pickle("./problem1_answer/answer_valid_list.pkl")
if "answer_train_y" not in locals():
    answer_train_y = open_pickle("./problem1_answer/answer_train_y.pkl")
if "answer_valid_y" not in locals():
    answer_valid_y = open_pickle("./problem1_answer/answer_valid_y.pkl")

if  answer_train_list == train_list and\
    answer_valid_list == valid_list and\
    answer_train_y == train_y and\
    answer_valid_y == valid_y :
    print("Problem #1 Solved")

Problem #1 Solved


In [11]:
from sklearn.preprocessing import MinMaxScaler

#TODO2
scaler = MinMaxScaler()

scaler.fit(train_list)
train_X = scaler.transform(train_list)

scaler_ = MinMaxScaler()

scaler_.fit(valid_list)
valid_X = scaler.transform(valid_list)

In [12]:
if "answer_train_X" not in locals():
    answer_train_X = open_pickle("./problem2_answer/answer_train_X.pkl")
if "answer_valid_X" not in locals():
    answer_valid_X = open_pickle("./problem2_answer/answer_valid_X.pkl")

if  np.array_equal(answer_train_X, train_X) and\
    np.array_equal(answer_valid_X, valid_X):
    print("Problem #2 Solved")

Problem #2 Solved


In [13]:
#학습을 위한 데이터 타입 변환
from BCE_model import train, test

train_X = train_X.astype(np.float32)
valid_X = valid_X.astype(np.float32)
train_y = np.array(train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

In [14]:
# train 데이터 학습 및 valid 데이터 예측
model = train(train_X, train_y)
predict = test(valid_X, valid_y, model)

  0%|          | 0/32 [00:00<?, ?it/s]

In [15]:
predict = np.array(predict, dtype="float32")

In [16]:
# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_predict = np.round(predict)

In [17]:
# 플로우 기준 이상 탐지 성능 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("confusion_matrix\n", confusion_matrix(round_valid_y, round_predict))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_predict): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_predict): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_predict): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_predict): .5f}")

confusion_matrix
 [[186414   3859]
 [  1797 133490]]
accuracy	:  0.98263
precision	:  0.97190
recall		:  0.98672
F-1		:  0.97925


In [18]:
# 정상 외부 IP set
outer_benignIP_set = open_pickle("./outer_benign_IP.pkl")
# 악성 외부 IP set
outer_malIP_set = open_pickle("./outer_mal_IP.pkl")

In [19]:
#외부 IP의 악성, 정상 개수 확인
len(outer_benignIP_set), len(outer_malIP_set)

(590, 457)

In [20]:
# Problem #3을 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [ ]:
#TODO3
from collections import defaultdict

theta = 0.1 # example

# 정답을 위한 것
predictIP_dict = defaultdict()
# 외부 IP별로 dataframe의 전체 flow를 담을 사전 
groupIP_dict = defaultdict(list)

# 기본 외부 IP 지정
for i in valid_df['Src IP']:
    if i in labelIP_dict:
        predictIP_dict[i] = 0

for i in valid_df['Dst IP']:
    if i in labelIP_dict:
        predictIP_dict[i] = 0

# 이상값 변화, IP별 그룹핑, 각 IP값에 해당 flow가 추가될 것임
for i, pred in enumerate(predict):
    if valid_df['Src IP'][i] in labelIP_dict:
        if pred >= theta:
            predictIP_dict[valid_df['Src IP'][i]] = 1
        groupIP_dict[valid_df['Src IP'][i]].append(valid_df.loc[i])
    if valid_df['Dst IP'][i] in labelIP_dict:
        if pred >= theta:
            predictIP_dict[valid_df['Dst IP'][i]] = 1
        groupIP_dict[valid_df['Dst IP'][i]].append(valid_df.loc[i])

# 빠지는 케이스 있는지 검증
print(len(valid_df))
cnt = 0
for val in groupIP_dict.values():
    for indi in val:
        cnt += 1
print(cnt)
print(len(groupIP_dict.keys()))

In [22]:
if "answer_predictIP_dict" not in locals():
    answer_predictIP_dict = open_pickle("./problem3_answer/answer_predictIP_dict.pkl")

if  predictIP_dict == answer_predictIP_dict:
    print("Problem #3 Solved")

Problem #3 Solved


In [23]:
import pickle

with open("./answer_predictIP_dict_new.pkl", 'wb') as f:
    pickle.dump(predictIP_dict, f)

In [24]:
# 정답 IP dictionary와 예측 IP dictionary를 비교하여 성능 평가
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(predictIP_dict[k])
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[380 210]
 [  5 452]]
accuracy	:  0.79465
precision	:  0.68278
recall		:  0.98906
F-1		:  0.80786
